In [5]:
from qiskit import *
from qiskit.circuit.library import GroverOperator
from qiskit.quantum_info import Statevector

In [9]:
from qiskit.visualization import plot_histogram

In [6]:
import numpy as np
from math import ceil

In [2]:
class GroverQLearner:
    ''' 
    Implement a quanutm reinforcement learning agent based on Grover amplitute enhancement and QLearing algorithm.

    Assumption:
    The dimensions of the state space and action space are both finite

    Parameters:
    env: the environment to solve; default is OpenAI gym "FrozenLake"
    state (int): current state
    action (int): current action 
    state_dimension (int): dimension of the state space
    action_dimension (int): dimension of the action space
    action_qregister_size (int): number of qubits on the quantum register for storing the action wavefunction 
    max_grover_length (int): maximum of the length of the grover iteration
    Q_values (2D np array): Q values of all (state, action) combinations; shape = (state_dimension, action_dimention)
    grover_lengths (2D np array): lengths of grover iterations of all (state, action) combinaitons; shape = (state_dimension, action_dimention)
    grover_operators (1D np array): grover_operators for all actions; grover_operators[a] records the grover operator constructed from action eigenfucntion a 
    action_circuits (1D np array): action quantum circuits for all states; action_circuits[s] records the quanutm circuit encoding the up-to-date action wavefunction of state s
    hyperparameters (dict): hyperparameters of learning; 
                            {
                                'k': prefactor of max grover length, 
                                'alpha': learning rate, 'gamma': discount, 
                                'epsilonr': tolerance of the Q values,
                                'max_epochs': max number of epochs for training,
                                'max_steps': max number of steps in every epoch
                            }
    backend: machine to execute the quanutm circuit jobs; could be either a simulator or a true quantum computer
    '''
    def __init__(self, env) -> None:
        self.env = env
        self.state = env.reset()
        self.action = 0
        self.state_dimension = env.observation_space.n
        self.action_dimension = env.action_space.n
        self.action_qregister_size = ceil(np.log2(self.action_dimension))
        self.max_grover_length = int(round(np.pi / (4 * np.arcsin(1. / np.sqrt(2 ** self.action_qregister_size))) - 0.5))
        self.Q_values = np.zeros((self.state_dimension, self.action_dimension), dtype=float) 
        self.grover_length = np.zeros((self.state_dimension, self.action_dimension), dtype=int)
        self.max_grover_length_reached = np.zeros((self.state_dimension, self.action_dimension), dtype=bool)
        self.grover_operators = self._init_grover_operators()
        self.action_circuits = self._init_action_circuits()
        self.hyperparameters = {
            'k': 0.1,
            'alpha': 0.05,
            'gamma': 0.99,
            'eps': 0.01,
            'max_epochs': 1000,
            'max_steps': 100
        }
        self.backend = Aer.get_backend('qasm_simulator')
            
    # hyperparameter setter
    def set_hyperparameters(self, params):
        pass

    def _init_action_circuits(self):
        '''
        Initialize the quanutm circuits encoding the action wavefunction of every state. Every initial action wavefunction is a equally weighted superposition of all action eignenfucntions. 
        '''
        action_circuits = np.zeros(self.state_dimension)
        for s in range(self.state_dimension):
            action_circuits[s] = QuantumCircuit(self.action_qregister_size, name='action_s{}'.format(s)) # construct the quanutm circuit
        for circuit in action_circuits:
            circuit.h(list(range(self.action_qregister_size))) # apply H gate to every qubit register to create the equally weighted superposition
        return action_circuits

    def _init_grover_operators(self):
        ''' 
        Initialize the grover operators of every action. U_grover := U_a0 * Ua where a0 is the equally superposition of all action eigenfunctions and a is an action eigenfunction. In fact,
        U_grover is not updated during the training process within the scope of this project.
        '''
        grove_operators = np.zeros(self.action_dimension)
        target_states = np.zeros(self.action_dimension)
        for i in range(self.action_dimension):
            state_binary = format(i, '0{}b'.format(self.action_qregister_size)) # generate the statevector binary string for encoding every action using the quantum register
            grove_operators[i] = GroverOperator(oracle=Statevector.from_label(state_binary)).to_instruction()
        return grove_operators

    def _take_action(self):
        ''' 
        Take an action under the current state by measuring the corresponding action wavefunction
        '''
        circuit = self.action_circuits[self.state] # the quanutm circuit encoding the up-to-date action wavefunction of the current state
        circuit_to_measure = circuit.copy() # make a copy of the circuit for measurement so that the original circuit is not broken by the measurement
        circuit_to_measure.measure_all() # take the action by measuring the current action wavefunciton
        job = execute(circuit_to_measure, backend=self.backend, shots=1) # execute the circuit using the backend
        result = job.result()
        counts = result.get_counts()
        action = int((list(counts.keys()))[0], 2) # take the action with highest probablity
        return action

    def _get_grover_length(self, reward, new_state):
        ''' 
        Calculate length of the Grover iteration after taking an action
        '''
        k = self.hyperparameters['k']
        return int(k * (reward + np.max(self.Q_values[new_state]))) # here we use max(Q_value[new_state]), it is also possible to use the expectation of Q_value[new_state] based on Born's rule 
        
    def _run_grover_iterations(self):
        '''
        Run grover iterations within one learning step
        '''
        length = self.grover_length(self.state, self.action) # number of grover operators(iterations) to append in this steps
        circuit = self.action_circuits(self.state) # the up-to-date quanutm circuit encodeing the action of current state
        grover_operator = self.grover_operators(self.action) # the grover operator of current action

    def _update_Q_values(self, reward, new_state):
        alpha = self.hyperparameters['alpha']
        gamma = self.hyperparameters['gamma']
        self.Q_value[self.state, self.action] = self.Q_values[self.state, self.action] + alpha * (reward + gamma * np.max(self.Q_value[new_state]) - self.Q_values[self.state, self.action])

    # train max epochs
    def train(self):
        pass

    